In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sklearn
import math
import joblib
from matplotlib import pyplot as plt

In [2]:
# Load the npz file
train_1 = np.load("data/train_data.npy")
train_2 = np.load("data/train_labels.npy")
train_df = DataFrame(train_1)
del train_1
train_lb_df = DataFrame(train_2)
del train_2

# For validation
val_1 = np.load("data/eval_data.npy")
val_2 = np.load("data/eval_labels.npy")
val_df = DataFrame(val_1)
del val_1
val_lb_df = DataFrame(val_2)
del val_2

# for unit_test

uni_df = train_df.iloc[100:,:]
uni_lb_df = train_lb_df.iloc[100:,:]

### To see the features importance
By random forest algo

In [3]:
# Load the forest
import joblib
forest = joblib.load( "Model/02_RF_training_300_0pt7" )
ft_imp = pd.DataFrame(forest.feature_importances_)
ft_imp.index = train_df.columns
ft_imp = ft_imp.sort_values(0, ascending=False)
ft_imp["feature_name"] = ft_imp.index
ft_imp["importance"] = ft_imp.iloc[:,0]
ft_imp = ft_imp[ ["feature_name", "importance"] ]
for i in range(len(ft_imp)):
    print("%s %0.6f" % (str(ft_imp.iloc[i,0]), ft_imp.iloc[i,1]))
    #print(ft_imp.iloc[i,0])
ft_imp.info()

24 0.185934
49 0.043888
0 0.018761
50 0.018217
52 0.017221
54 0.016561
8 0.015748
6 0.015664
63 0.015401
43 0.015201
9 0.014880
53 0.014763
48 0.014364
18 0.014254
47 0.013960
46 0.013954
28 0.013883
21 0.013323
13 0.013307
19 0.013262
20 0.013218
25 0.013091
22 0.012969
7 0.012937
66 0.012907
4 0.012872
23 0.012868
41 0.012827
35 0.012590
39 0.012398
37 0.012393
29 0.012334
68 0.012291
10 0.012225
26 0.012144
45 0.012039
11 0.011961
38 0.011806
34 0.011662
32 0.011610
30 0.011327
36 0.011323
51 0.011281
67 0.011272
27 0.011267
62 0.011215
69 0.011161
44 0.011099
64 0.011059
33 0.010841
61 0.010561
16 0.010168
42 0.010017
31 0.009997
65 0.009834
1 0.009415
12 0.009379
40 0.009338
14 0.009186
2 0.008308
15 0.008159
3 0.008142
17 0.007996
5 0.005181
58 0.003286
55 0.002541
57 0.001897
59 0.001812
60 0.001009
56 0.000243
<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 24 to 56
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        -----

### Abandon the feature with < 0.005 importance

In [4]:
imp_notdrop_list = [ x for x in range(70) if x not in range(55,61) ]
train_df = train_df[imp_notdrop_list]
val_df = val_df[imp_notdrop_list]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174461 entries, 0 to 1174460
Data columns (total 64 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1174461 non-null  float64
 1   1       1174461 non-null  float64
 2   2       1174461 non-null  float64
 3   3       1174461 non-null  float64
 4   4       1174461 non-null  float64
 5   5       1174461 non-null  float64
 6   6       1174461 non-null  float64
 7   7       1174461 non-null  float64
 8   8       1174461 non-null  float64
 9   9       1174461 non-null  float64
 10  10      1174461 non-null  float64
 11  11      1174461 non-null  float64
 12  12      1174461 non-null  float64
 13  13      1174461 non-null  float64
 14  14      1174461 non-null  float64
 15  15      1174461 non-null  float64
 16  16      1174461 non-null  float64
 17  17      1174461 non-null  float64
 18  18      1174461 non-null  float64
 19  19      1174461 non-null  float64
 20  20      1174461 non-null

In [5]:
# To compare i and j
def Training_EnsembleModel(X, y, i=0, j=1, name="" ):
    
    #X = train_df.loc[((train_lb_df[0]==i)|(train_lb_df[0]==j)) & (train_lb_df.index>1000000),:]
    #y = train_lb_df.loc[((train_lb_df[0]==i)|(train_lb_df[0]==j)) & (train_lb_df.index>1000000),0]
    
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import VotingClassifier
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import make_pipeline
    from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
    
    lr = LogisticRegression(C=0.001)
    pipeline = make_pipeline(StandardScaler(), lr)
    
    forest = RandomForestClassifier( n_estimators=300, max_features=0.7, criterion='gini', random_state=22, n_jobs=-1 )
    
    tree = DecisionTreeClassifier( random_state=1, max_depth=18, max_features="sqrt", max_leaf_nodes=None )
    ABC = AdaBoostClassifier( base_estimator=tree, random_state=11, learning_rate=0.1, n_estimators=50 )
    
    model = VotingClassifier(estimators=[
            ('lr', pipeline), ('rf', forest), ('bdt', ABC)],
            voting='soft')
    model = model.fit(X, y)
    joblib.dump( model, "02_final_Ensemble_training_afterCut_" + str(i) + "_" + str(j) + "_" + name )
    print("Accuracy : ", accuracy_score(val_lb_df.loc[(val_lb_df[0]==i)|(val_lb_df[0]==j),:],  model.predict(val_df.loc[(val_lb_df[0]==i)|(val_lb_df[0]==j),:])))

In [6]:
par_02 = [0.001, 300, 0.7, 50, 0.1, 18, "sqrt", None]
Training_EnsembleModel(train_df.loc[((train_lb_df[0]==0)|(train_lb_df[0]==2)) & (train_lb_df.index>1000000),:]
                       , train_lb_df.loc[((train_lb_df[0]==0)|(train_lb_df[0]==2)) & (train_lb_df.index>1000000),0]
                       , 0, 2)

Accuracy :  0.7007275471437378
